In [ ]:
# Testing Cell
import aviary.api as av
from aviary.utils.doctape import glue_variable

glue_variable('fuel_burned', av.Mission.Summary.FUEL_BURNED, True)

# Optimizing the Mission Profile of a Conventional Aircraft

Building upon our previous example, this notebook introduces more complexity into the Aviary optimization process.
Please see the [simple mission example](simple_mission_example) if you haven't already.

## Increasing Complexity in Phase Information

We will now modify the `phase_info` object from our prior example by increasing {glue:md}`num_segments` to 3 and setting {glue:md}`mach_optimize` to `True` in each of the three phases.
This means that we'll query the aircraft performance at more points along the mission and also give the optimizer the freedom to choose an optimal Mach profile.

```{note}
We are still using a {glue:md}`mach_polynomial_order` and {glue:md}`altitude_polynomial_order` of 1, which means that the optimal Mach profiles for each phase will be linear (straight lines).
Later in this example, we increase this order which will allow the optimizer to choose a more complex Mach profile.
```

In [ ]:
phase_info = {
    'pre_mission': {'include_takeoff': False, 'optimize_mass': True},
    'climb_1': {
        'subsystem_options': {'core_aerodynamics': {'method': 'computed'}},
        'user_options': {
            'num_segments': 3,
            'order': 3,
            'distance_solve_segments': False,
            'mach_optimize': True,
            'mach_polynomial_order': 1,
            'mach_initial': (0.2, 'unitless'),
            'mach_final': (0.72, 'unitless'),
            'mach_bounds': ((0.18, 0.74), 'unitless'),
            'altitude_optimize': False,
            'altitude_polynomial_order': 1,
            'altitude_initial': (0.0, 'ft'),
            'altitude_final': (30500.0, 'ft'),
            'altitude_bounds': ((0.0, 31000.0), 'ft'),
            'throttle_enforcement': 'path_constraint',
            'time_initial_bounds': ((0.0, 0.0), 'min'),
            'time_duration_bounds': ((27.0, 81.0), 'min'),
        },
        'initial_guesses': {'time': ([0, 54], 'min')},
    },
    'cruise': {
        'subsystem_options': {'core_aerodynamics': {'method': 'computed'}},
        'user_options': {
            'num_segments': 3,
            'order': 3,
            'mach_optimize': True,
            'mach_polynomial_order': 1,
            'mach_initial': (0.72, 'unitless'),
            'mach_final': (0.72, 'unitless'),
            'mach_bounds': ((0.7, 0.74), 'unitless'),
            'altitude_optimize': False,
            'altitude_initial': (30500.0, 'ft'),
            'altitude_final': (31000.0, 'ft'),
            'altitude_bounds': ((30000.0, 31500.0), 'ft'),
            'throttle_enforcement': 'boundary_constraint',
            'time_initial_bounds': ((27.0, 81.0), 'min'),
            'time_duration_bounds': ((85.5, 256.5), 'min'),
        },
        'initial_guesses': {'time': ([54, 171], 'min')},
    },
    'descent_1': {
        'subsystem_options': {'core_aerodynamics': {'method': 'computed'}},
        'user_options': {
            'num_segments': 3,
            'order': 3,
            'mach_optimize': True,
            'mach_polynomial_order': 1,
            'mach_initial': (0.72, 'unitless'),
            'mach_final': (0.2, 'unitless'),
            'mach_bounds': ((0.18, 0.74), 'unitless'),
            'altitude_optimize': False,
            'altitude_initial': (31000.0, 'ft'),
            'altitude_final': (500.0, 'ft'),
            'altitude_bounds': ((0.0, 31500.0), 'ft'),
            'throttle_enforcement': 'path_constraint',
            'time_initial_bounds': ((112.5, 337.5), 'min'),
            'time_duration_bounds': ((26.5, 79.5), 'min'),
        },
        'initial_guesses': {'time': ([225, 53], 'min')},
    },
    'post_mission': {
        'include_landing': False,
        'constrain_range': True,
        'target_range': (1915, 'nmi'),
    },
}

In [ ]:
# Testing Cell
from aviary.models.missions.height_energy_default import phase_info as HE_phase_info
from aviary.utils.doctape import glue_keys

HE_phase_info.update(phase_info)
glue_keys(HE_phase_info)

## Running Aviary with Updated Parameters

Let's run the Aviary optimization with our updated `phase_info` object in the same way as before.


In [ ]:
from openmdao.core.problem import _clear_problem_names

_clear_problem_names()  # need to reset these to simulate separate runs
from openmdao.utils.reports_system import clear_reports

clear_reports()

In [ ]:
import aviary.api as av

prob = av.run_aviary(
    'models/aircraft/test_aircraft/aircraft_for_bench_FwFm.csv',
    phase_info,
    optimizer='SLSQP',
    make_plots=True,
)

Now that we've run Aviary, we can look at the results.
Open up the automatically generated `traj_results_report.html` and scroll through it to visualize the results.

Here are the altitude and Mach profiles:

![Altitude and Mach Profiles](images/advanced_results.png)

We note two major changes compared to our first example.

The first is that we have many more points where the flight dynamics were evaluated because we increased {glue:md}`num_segments` to 3.
This means that we have more points shown on the resulting plots.

The second is that the optimizer chose the optimal Mach profile.
Again, each phase's Mach profile is constrained to be linear because we set {glue:md}`mach_polynomial_order` and {glue:md}`altitude_polynomial_order` to 1.
However, we see that the optimizer chose to decrease the Mach number during the cruise-climb segment to minimize fuel burn.

```{note}
Remember, we did not allow the optimizer to control the altitude profile, so that remains fixed.
```

Let's take a look at the optimization objective, {glue:md}`fuel_burned`:

In [ ]:
print(prob.get_val(av.Mission.Summary.FUEL_BURNED, units='kg')[0])

We can print `fuel_burned` in pounds easily, thanks to OpenMDAO's automatic unit conversion feature:

In [ ]:
print(prob.get_val(av.Mission.Summary.FUEL_BURNED, units='lb')[0])

In [ ]:
# Testing Cell
from aviary.utils.doctape import glue_variable

new_filename = 'modified_aircraft.csv'
glue_variable(new_filename, md_code=True)

## Modifying the Aircraft Configuration

Next, we'll modify the aircraft configuration by decreasing the wing aspect ratio by 0.2.
This results in a less slender wing, which will increase the induced drag.
We've made this change and have a modified aircraft data file called {glue:md}`modified_aircraft.csv`.

In [ ]:
import csv

filename = 'models/aircraft/test_aircraft/aircraft_for_bench_FwFm.csv'
filename = av.get_path(filename)

# Read the file
with open(filename, 'r') as file:
    reader = csv.reader(file)
    lines = list(reader)

# Find the index of the line containing 'aircraft:wing:span'
index = None
for i, line in enumerate(lines):
    if 'aircraft:wing:aspect_ratio' in line:
        index = i
        break

# Modify the value in the line
if index is not None:
    aspect_ratio = float(lines[index][1]) - 0.2
    lines[index][1] = str(aspect_ratio)

# Write the modified content to a new CSV file
with open(new_filename, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(lines)

## Re-running the Optimization with Modified Aircraft

Now, let's re-run the optimization with the modified aircraft configuration.
We'll use the same `phase_info` object as before, but we'll change the input deck to point to our new aircraft file.

In [ ]:
from openmdao.core.problem import _clear_problem_names

_clear_problem_names()  # need to reset these to simulate separate runs
from openmdao.utils.reports_system import clear_reports

clear_reports()

In [ ]:
prob = av.run_aviary('modified_aircraft.csv', phase_info, optimizer='SLSQP', make_plots=True)

The case again converged in relatively few iterations.
Let's take a look at the fuel burn value:

In [ ]:
print(prob.get_val(av.Mission.Summary.FUEL_BURNED, units='kg')[0])

As expected, it's a bit higher than our prior run that had a larger aspect ratio.

## Increasing the Polynomial Control Order

Next, we'll increase the {glue:md}`mach_polynomial_order` and {glue:md}`altitude_polynomial_order` to 3 for the climb and descent phases.
This means that the optimizer will be able to choose a cubic Mach and altitude profile per phase instead of a straight line.
We'll use the original aircraft configuration for this run.

```{note}
We'll use the IPOPT optimizer for this problem as it will handle the increased complexity better than SLSQP.
```

In [ ]:
from openmdao.core.problem import _clear_problem_names

_clear_problem_names()  # need to reset these to simulate separate runs
from openmdao.utils.reports_system import clear_reports

clear_reports()

In [ ]:
phase_info['climb_1']['user_options']['mach_polynomial_order'] = 3
phase_info['climb_1']['user_options']['altitude_polynomial_order'] = 3
phase_info['cruise']['user_options']['mach_polynomial_order'] = 1
phase_info['cruise']['user_options']['altitude_polynomial_order'] = 1
phase_info['descent_1']['user_options']['mach_polynomial_order'] = 3
phase_info['descent_1']['user_options']['altitude_polynomial_order'] = 3

prob = av.run_aviary(
    'models/aircraft/test_aircraft/aircraft_for_bench_FwFm.csv',
    phase_info,
    optimizer='IPOPT',
    make_plots=True,
)

And let's print out the objective value, fuel burned:

In [ ]:
print(prob.get_val(av.Mission.Summary.FUEL_BURNED, units='kg')[0])

The added flexibility in the mission allowed the optimizer to reduce the fuel burn compared to the linear Mach profile case.

Looking at the altitude and Mach profiles, we see that the optimizer chose a more subtly complex Mach profile:

![Altitude and Mach Profiles](images/cubic_advanced_results.png)

## Conclusion

This example demonstrated how to use Aviary to optimize a more complex mission.
We increased the number of segments in the mission, allowed the optimizer to choose the optimal Mach profile, and increased the polynomial control order to allow for more complex Mach profiles.
We also modified the aircraft configuration to demonstrate how Aviary can be used to quickly evaluate the impact of design changes on the mission performance.
